<a href="https://colab.research.google.com/github/ScriptSherpa/TrackCount360-YOLO-Based-Object-Tracking-Directional-Counting-System/blob/main/detect%20all%20classes%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 📦 Install dependencies
!pip install -q ultralytics opencv-python-headless

# 📚 Import libraries
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import files
import os

# 📤 Upload video
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# 🧠 Load YOLOv8n model
model = YOLO("yolov8n.pt")  # Lightweight and fast

# 🎬 Track the drone in the uploaded video
results = model.track(
    source=video_path,
    persist=True,                # Keep IDs across frames
    imgsz=480,                   # Resize for speed
    conf=0.25,                   # Confidence threshold
    tracker="bytetrack.yaml",   # ByteTrack for multi-object tracking
    save=True                    # Save annotated video
)

# 🔍 Find output file path
import glob

output_dir = "runs/track/predict/"
output_files = glob.glob(os.path.join(output_dir, "*.mp4"))
output_path = output_files[0] if output_files else None

# ⬇️ Download the output video
if output_path:
    print(f"\n✅ Tracking complete! Video saved at: {output_path}")
    files.download(output_path)
else:
    print("❌ Output video not found.")


Saving WhatsApp Video 2025-04-12 at 17.27.57.mp4 to WhatsApp Video 2025-04-12 at 17.27.57 (1).mp4


100%|██████████| 6.25M/6.25M [00:00<00:00, 51.6MB/s]


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.0s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/419) /content/WhatsApp Video 2025-04-12 at 17.27.57 (1).mp4: 480x288 9 persons, 1 bench, 90.3ms
video 1/1 (frame 

In [ ]:
# Install necessary packages
!pip install ultralytics opencv-python-headless opencv-contrib-python-headless

# Import required libraries
from ultralytics import YOLO
import cv2
import numpy as np
from sort import Sort

# Initialize video capture
cap = cv2.VideoCapture("../Videos/people.mp4")

# Load the YOLOv8 model
model = YOLO("../Yolo-Weights/yolov8l.pt")

# Define class names
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# Initialize the SORT tracker
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# Define line limits for counting
limitsUp = [103, 161, 296, 161]
limitsDown = [527, 489, 735, 489]

# Initialize counters
totalCountUp = []
totalCountDown = []

while True:
    success, img = cap.read()
    if not success:
        break

    # Perform inference
    results = model(img, stream=True)
    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Confidence
            conf = box.conf[0].item()
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            if currentClass == "person" and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # Update tracker
    resultsTracker = tracker.update(detections)

    # Draw lines
    cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 0, 255), 5)
    cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 0, 255), 5)

    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1

        # Draw bounding box and ID
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        cv2.putText(img, f'ID: {int(id)}', (x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 3)

        # Center point
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), -1)

        # Count vehicles crossing the lines
        if limitsUp[0] < cx < limitsUp[2] and limitsUp[1] - 15 < cy < limitsUp[1] + 15:
            if id not in totalCountUp:
                totalCountUp.append(id)
                cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 255, 0), 5)

        if limitsDown[0] < cx < limitsDown[2] and limitsDown[1] - 15 < cy < limitsDown[1] + 15:
            if id not in totalCountDown:
                totalCountDown.append(id)
                cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 255, 0), 5)

    # Display counts
    cv2.putText(img, f'Up: {len(totalCountUp)}', (50, 50), cv2.FONT_HERSHEY_PLAIN, 5, (0, 255, 0), 3)
    cv2.putText(img, f'Down: {len(totalCountDown)}', (50, 100), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 255), 3)

    # Show the image
    cv2_imshow(img)
    cv2.waitKey(1)
